In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import requests
from tqdm import tqdm

In [440]:
load_dotenv()

True

In [441]:
# Paramètres de connexion MySQL
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
database = os.getenv('DB_NAME')

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

### Valeurs foncieres

In [483]:
# Chargement du fichier excel
valeurs_foncieres = pd.read_excel("../data/valeurs_foncieres.xlsx")

# Nettoyage des noms de colonnes
valeurs_foncieres.columns = valeurs_foncieres.columns.str.strip()
valeurs_foncieres.columns = valeurs_foncieres.columns.str.lower()
valeurs_foncieres.columns = valeurs_foncieres.columns.str.replace(' ', '_')

valeurs_foncieres = valeurs_foncieres[['identifiant_local', 'date_mutation', 'valeur_fonciere', 'no_voie',
                                       'type_de_voie', 'voie', 'nombre_pieces_principales', 'surface_reelle_bati',
                                       'surface_carrez_du_1er_lot', 'type_local', 'code_commune']].copy()

# Renommer pour se coller au schema
valeurs_foncieres.rename(columns={
    'identifiant_local': 'id_bien',
    'date_mutation': 'date',
    'valeur_fonciere': 'valeur',
    'no_voie': 'no_voie',
    'type_de_voie': 'type_voie',
    'voie': 'voie',
    'nombre_pieces_principales': 'total_piece',
    'surface_reelle_bati': 'surface_local',
    'surface_carrez_du_1er_lot': 'surface_carrez',
    'type_local': 'type_local',
    'code_commune': 'id_commune'
}, inplace=True)

# Problème de fichier => identifiant_local nul
valeurs_foncieres['id_bien'] = range(1, len(valeurs_foncieres) + 1)


# Convertir la date et filtrer le 1er semestre 2020 pour respecter le compte-rendu de réunion de validation
valeurs_foncieres['date'] = pd.to_datetime(valeurs_foncieres['date'], errors='coerce')
valeurs_foncieres = valeurs_foncieres[valeurs_foncieres['date'].dt.year == 2020]
valeurs_foncieres = valeurs_foncieres[valeurs_foncieres['date'].dt.month <= 6]

# Remplacement des valeurs manquantes
valeurs_foncieres['surface_local'] = valeurs_foncieres['surface_local'].fillna(0)
valeurs_foncieres['total_piece'] = valeurs_foncieres['total_piece'].fillna(0)
valeurs_foncieres['surface_carrez'] = valeurs_foncieres['surface_carrez'].fillna(0)
valeurs_foncieres['type_voie'] = valeurs_foncieres['type_voie'].fillna('INCONNU')
valeurs_foncieres['voie'] = valeurs_foncieres['voie'].fillna('INCONNU')
valeurs_foncieres['type_local'] = valeurs_foncieres['type_local'].fillna('INCONNU')

valeurs_foncieres.dropna(subset=['date', 'valeur', 'id_commune'], inplace=True)

C:\Users\moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Regions + Communes

In [443]:
regions_list = [
    "Auvergne-Rhône-Alpes",
    "Bourgogne-Franche-Comté",
    "Bretagne",
    "Centre-Val de Loire",
    "Corse",
    "Grand Est",
    "Hauts-de-France",
    "Île-de-France",
    "Normandie",
    "Nouvelle-Aquitaine",
    "Pays de la Loire",
    "Provence-Alpes-Côte d'Azur",
    "La Réunion"
]

regions_df = pd.DataFrame({'nom_region': regions_list})

In [ ]:
# Chargement du fichier excel
communes = pd.read_excel("../data/donnees_communes.xlsx")

communes.columns = communes.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')

# Corriger les problèmes d'encodage des noms de régions
communes['reg'] = communes['reg'].apply(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)
communes['com'] = communes['com'].apply(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)

regions_table_df = pd.DataFrame({
    'nom_region': [
        "Auvergne-Rhône-Alpes", "Bourgogne-Franche-Comté", "Bretagne", "Centre-Val de Loire", "Corse",
        "Grand Est", "Hauts-de-France", "Île-de-France", "Normandie", "Nouvelle-Aquitaine",
        "Pays de la Loire", "Provence-Alpes-Côte d'Azur", "La Réunion"
    ],
    'id_region': range(1, 14)
})

# Fusionner les régions sur la base du nom de la région
communes = communes.merge(
    regions_table_df[['nom_region', 'id_region']],
    left_on='reg',
    right_on='nom_region',
    how='left'
)

communes = communes[['codcom', 'id_region', 'coddep', 'com']].rename(columns={
    'codcom': 'id_commune',
    'coddep': 'code_departement',
    'com': 'nom_commune'
})

communes = communes.dropna(subset=['id_region'])
communes['id_region'] = communes['id_region'].astype(int)

### Populations

In [445]:
def get_population_by_dep(code_departement, nom_commune):
    url = f"https://geo.api.gouv.fr/departements/{code_departement}/communes"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Lève une erreur si la requête a échoué

        nom_commune = nom_commune.strip()

        data = response.json()
        for commune in data:
            if str(commune['codeDepartement']) == str(code_departement) and commune['nom'] == nom_commune:
                return commune['population']

        return None

    except requests.exceptions.RequestException as e:
        return None

    except Exception as e:
        return None


In [448]:
# Chargement du fichier excel
populations = pd.read_excel("../data/populations_referentiel_geograpgique.xlsx")

populations.columns = populations.columns.str.strip().str.lower().str.replace(' ', '_')

populations = populations[['com_code','dep_nom', 'dep_code', 'uucr_nom']].copy()
populations.rename(columns={
    'com_code': 'id_commune',
}, inplace=True)

populations['code_departement'] = populations['dep_code']

populations['dep_nom'] = populations['dep_nom'].str.encode('utf-8').str.decode('utf-8')
populations['dep_nom'] = populations['dep_nom'].apply(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)

populations['nom_commune'] = populations['uucr_nom'].str.encode('utf-8').str.decode('utf-8')
populations['nom_commune'] = populations['nom_commune'].apply(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)

populations = populations[['id_commune', 'nom_commune','dep_nom', 'code_departement', ]].copy()

In [449]:
tqdm.pandas(disable=True)
populations['population'] = populations.progress_apply(
    lambda x: get_population_by_dep(x['code_departement'], x['nom_commune']),
    axis=1
)

In [450]:
# Les données du gouvernement sont issus datées de 2025
populations['annee'] = 2025

In [ ]:
populations = populations.merge(
    communes[['id_commune', 'nom_commune']],
    how='left',
    left_on='nom_commune',
    right_on='nom_commune'
)

populations.drop(columns=['id_commune_x'], inplace=True)
populations.rename(columns={'id_commune_y': 'id_commune'}, inplace=True)
populations.dropna(subset=['id_commune'], inplace=True)
populations['id_commune'] = populations['id_commune'].astype(int)


### Insertion des données dans la DB

In [485]:
# Insérer les données dans MySQ
print("Début de l'importation des données")

# Region
regions_df.to_sql('region', con=engine, if_exists='append', index=False)
print("Données region insérées")

# Commune
communes = communes[['id_commune', 'id_region', 'code_departement', 'nom_commune']]
communes = communes.drop_duplicates(subset='id_commune')
communes.to_sql('commune', con=engine, if_exists='append', index=False)
print("Données 'commune' insérées")

# Bien
biens = valeurs_foncieres[['id_bien', 'no_voie', 'type_voie', 'voie', 'total_piece', 'surface_carrez', 'surface_local', 'type_local', 'id_commune']].drop_duplicates()
biens.to_sql("bien", con=engine, if_exists='append', index=False)
print("Données 'bien' insérées")

# Vente
ventes = valeurs_foncieres[['id_bien', 'date', 'valeur']].copy()
ventes.dropna(subset=['date', 'valeur'], inplace=True)
ventes.to_sql("vente", con=engine, if_exists='append', index=False)
print("Données 'vente' insérées")

# Population
populations = populations[['id_commune', 'annee', 'population']]
populations.to_sql("population", con=engine, if_exists='append', index=False)
print("Données 'population' insérées")

print("Importation terminée avec succès")

Début de l'importation des données
Données region insérées
Données 'commune' insérées
Données 'bien' insérées
Données 'vente' insérées
Données 'population' insérées
Importation terminée avec succès
